In [116]:
from typing import Tuple
import numpy as np

def generate_grid_mesh(height: int, width: int, flatten_point_array: bool = True) -> Tuple[np.ndarray, np.ndarray]:
  # define grid offsets
  grid_offsets = np.zeros((height + 1, width + 1, 2), dtype=np.int32)
  grid_offsets[:, :, 0] = np.arange(height + 1)[:, None]
  grid_offsets[:, :, 1] = np.arange(width + 1)[None, :]
  
  unique_grid_unit_points = np.array([[0.0, 0.0], [0.5, 0.0], [0.0, 0.5], [0.5, 0.5]])

  # h+1 x w+1 x 4 x 2
  grid_points = unique_grid_unit_points[None, None, :] + np.flip(grid_offsets, -1)[:, :, None, :]

  grid_indices = np.zeros((height + 1, width + 1, 4, 3), dtype=np.int32)
  grid_indices[:, :, :, 0] = np.arange(4)[None, None, :]
  grid_indices[:, :, :, 1:] = grid_offsets[:, :, None, :]

  # h x w x 4 x 3
  a = grid_indices[:-1, :-1]
  b = grid_indices[:-1, 1:]
  c = grid_indices[1:, :-1]
  d = grid_indices[1:, 1:]
  gridded_triangles = [
    (a[:, :, 0], a[:, :, 3], a[:, :, 1]),
    (a[:, :, 0], a[:, :, 2], a[:, :, 3]),
    (a[:, :, 1], a[:, :, 3], b[:, :, 0]),
    (a[:, :, 3], b[:, :, 2], b[:, :, 0]),
    (a[:, :, 2], c[:, :, 0], a[:, :, 3]),
    (a[:, :, 3], c[:, :, 0], c[:, :, 1]),
    (a[:, :, 3], c[:, :, 1], d[:, :, 0]),
    (a[:, :, 3], d[:, :, 0], b[:, :, 2])
  ]
  triangles = []
  for pointset in gridded_triangles:
    # h x w x 3 x 3 (third axis is the grid coordinate)
    pointset = np.stack(pointset, axis=-1)
    # h x w x 3 (grid coordinate converted to flat index)
    pointset = (
      pointset[:, :, 2, :] * 4
      + pointset[:, :, 1, :] * (width + 1) * 4
      + pointset[:, :, 0, :]
    )
    # (h x w) x 3 (grid of flat index triplets converted to list of triplets)
    pointset = pointset.reshape((-1, 3))
    triangles.append(pointset)
  # (8 x h x w) x 3 (list of triangles - 8 per grid face)
  triangles = np.concatenate(triangles)

  points = np.concatenate((grid_points, np.zeros((height + 1, width + 1, 4, 1))), axis=-1)
  if flatten_point_array:
    points = points.reshape((-1, 3))
  return points, triangles

points, triangles = generate_grid_mesh(4, 2)
save_stl(points, triangles, 'output.stl')

In [ ]:
from stl import mesh

def save_stl(vertices, faces, filename):
    """
    Save vertex and face data as an STL file.

    @param vertices: n x 3 array of 3D points.
    @param faces: m x 3 array of vertex indices specifying triangles.
    """
    # Create the mesh
    triangle_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    
    # Fill in the mesh vertices for each triangle
    for i, face in enumerate(faces):
        for j in range(3):
            triangle_mesh.vectors[i][j] = vertices[face[j]]
    
    # Save to file
    triangle_mesh.save(filename)

# Example usage
vertices = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

faces = np.array([
    [0, 1, 2],
    [0, 1, 3],
    [0, 2, 3],
    [1, 2, 3]
])

save_stl(points, triangles, 'output.stl')

In [ ]:
import os
os.chdir('C:/Users/clack/Projects/nwm')

from experiment.nav2d import Topo

state = Topo.random()
terrain = state.terrain[120:136, 120:136]
# h+1 x w+1 x 4 x 3
height = terrain.shape[0] * 2 - 1
width = terrain.shape[1] * 2 - 1
points, triangles = generate_grid_mesh(height, width, flatten_point_array=False)
grid_offsets = np.zeros((height + 1, width + 1, 2), dtype=np.int32)
grid_offsets[:, :, 0] = np.arange(height + 1)[:, None]
grid_offsets[:, :, 1] = np.arange(width + 1)[None, :]

even_x = grid_offsets[:, :, 0] % 2 == 0
even_y = grid_offsets[:, :, 1] % 2 == 0
unit_mask = even_x & even_y
x_sandwich_mask = ~even_x & even_y
y_sandwich_mask = even_x & ~even_y
corner_mask = ~even_x & ~even_y

modulated_terrain_height = np.zeros_like(terrain)
modulated_terrain_height[terrain < state.wall_height] = terrain[terrain < state.wall_height] * 10
modulated_terrain_height[terrain >= state.wall_height] = terrain[terrain >= state.wall_height] * 20 + 4
# set every other 3x3 grid region to the corresponding terrain height

points[unit_mask, :, 2] = modulated_terrain_height[:, :, None].repeat(4, -1).reshape(-1, 4)
points[x_sandwich_mask, 0, 2] = modulated_terrain_height[:, :].reshape(-1)
points[x_sandwich_mask, 1, 2] = modulated_terrain_height[:, :].reshape(-1)
points[y_sandwich_mask, 0, 2] = modulated_terrain_height[:, :].reshape(-1)
points[y_sandwich_mask, 2, 2] = modulated_terrain_height[:, :].reshape(-1)
points[corner_mask, 0, 2] = modulated_terrain_height[:, :].reshape(-1)


save_stl(points.reshape(-1, 3), triangles, 'output.stl')